In [2]:
# Basic Libraries
import numpy as np
import pandas as pd
import seaborn as sb
import matplotlib.pyplot as plt 
sb.set() 

In [3]:
final_data = pd.read_csv("final_data.csv")
final_data.drop(labels="Unnamed: 0", axis="columns", inplace=True)
final_data

FileNotFoundError: [Errno 2] No such file or directory: 'final_data.csv'

In [ ]:
#Extracting variables that could affect home goals
HomeData = pd.DataFrame(final_data[["HomePossession", "HomeTouches", "HomePasses", "HomeTackles", "HomeClearances", "HomeOffsides", "HomeShots", "HomeShotsOnTarget", "HomeGoals"]])
HomeData

# Model 1: Simple Multivariate Linear Regression to predict goals

#### **Response: HomeGoals**
####  **Predictors: Every other column in the cleaned HomeData data. See predictors below.**

In [4]:
Model_1_predictors = HomeData.drop(columns="HomeGoals")
Model_1_response = HomeData["HomeGoals"]

Model_1_predictors

NameError: name 'HomeData' is not defined

### Model Training

In [5]:
#Setting up the regresion problem with Train and Test datasets
#Change this to KFolds cross validation
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(Model_1_predictors, Model_1_response, random_state = 40, test_size = 0.25)

from sklearn.linear_model import LinearRegression

#Don't think should use regression

# Linear Regression using Train Data
linreg = LinearRegression()         # create the linear regression object
linreg.fit(X_train, y_train)        # train the linear regression model

# Coefficients of the Linear Regression line
print('Intercept of Regression \t: b = ', linreg.intercept_)
print()

for i,x in enumerate(["HomePossession", "HomeTouches", "HomePasses", "HomeTackles", "HomeClearances", "HomeOffsides", "HomeShots", "HomeShotsOnTarget"]):
  print('Coefficient of ' +x + ' (times 1000):\t: ', linreg.coef_[i]*1000)
print()

# Print the Coefficients against Predictors




NameError: name 'Model_1_predictors' is not defined

In [6]:
y_train_pred = linreg.predict(X_train)
y_test_pred = linreg.predict(X_test)

# Plot the Predictions vs the True values
f, axes = plt.subplots(1, 2, figsize=(24, 12))
axes[0].scatter(y_train, y_train_pred, color = "blue")
axes[0].plot(y_train, y_train, 'w-', linewidth = 1)
axes[0].set_xlabel("True number of goals scored (Train)")
axes[0].set_ylabel("Actual number of goals scored (Train)")
axes[1].scatter(y_test, y_test_pred, color = "green")
axes[1].plot(y_test, y_test, 'w-', linewidth = 1)
axes[1].set_xlabel("True number of goals scored (Test)")
axes[1].set_ylabel("Actual number of goals scored (Test)")
plt.show()

NameError: name 'linreg' is not defined

In [7]:
from sklearn.metrics import mean_squared_error

# Check the Goodness of Fit (on Train Data)
print("Goodness of Fit of Model \tTrain Dataset")
print("Explained Variance (R^2) \t:", linreg.score(X_train, y_train))
print("Mean Squared Error (MSE) \t:", mean_squared_error(y_train, y_train_pred))
print()

# Check the Goodness of Fit (on Test Data)
print("Goodness of Fit of Model \tTest Dataset")
print("Explained Variance (R^2) \t:", linreg.score(X_test, y_test))
print("Mean Squared Error (MSE) \t:", mean_squared_error(y_test, y_test_pred))
print()

Goodness of Fit of Model 	Train Dataset


NameError: name 'linreg' is not defined

## **Insights**

**Goal Prediction:**

Rather surprisingly, our model did not do too well - the explained variance only had an R^2 value of around 0.18 for predicting goals from the above data. For this, maybe it will be more useful to try another model. Lets try *Model 2: Random Forest* next to find a better way to predict goals!

**Feature importance:**

As we can see from the above model, the variable with the highest coefficient is Shots On Target. So our recommendation to football teams is to take more shots, and make sure that they are on target.

Well... that a bit obvious. If you take more shots (on target), you score more goals. Well, lets try to recommend something more interesting for teams. Lets try to re-formulate the problem into *problem 2*. 

But first, let's try our Random Forest model:

# Model 2: Random Forest

#### **Response: Same as Model 1**
#### **Predictors: Same as Model 1**

### Model Training

In [10]:
y = pd.DataFrame(HomeGoals.astype('category'))
f = plt.figure(figsize=(18, 6))
sb.countplot(y['HomeGoals'])

from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import GridSearchCV


predictors = ["HomePossession", "HomeTouches", "HomePasses", "HomeTackles", "HomeClearances", "HomeOffsides", "HomeShots", "HomeShotsOnTarget"]
X = pd.DataFrame(final_data[predictors])

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state = 42, test_size = 0.25)
rforest = GridSearchCV(RandomForestClassifier(max_depth = 4), {
  "n_estimators": [100, 200, 300],
},cv = 5) ##Can change the parameters here

rforest.fit(X_train, y_train.values.ravel()) 
df = pd.DataFrame(rforest.cv_results_)
y_train_pred = rforest.predict(X_train)
y_test_pred = rforest.predict(X_test)



NameError: name 'HomeGoals' is not defined

### Model Performance

In [11]:
# Check the Goodness of Fit (on Train Data)
print("Goodness of Fit of Model \tTrain Dataset")
print("Classification Accuracy \t:", rforest.score(X_train, y_train))
print()

# Check the Goodness of Fit (on Test Data)
print("Goodness of Fit of Model \tTest Dataset")
print("Classification Accuracy \t:", rforest.score(X_test, y_test))
print()

# Plot the Confusion Matrix for Train and Test
f, axes = plt.subplots(1, 2, figsize=(12, 4))
sb.heatmap(confusion_matrix(y_train, y_train_pred),
           annot = True, fmt=".0f", annot_kws={"size": 18}, ax = axes[0])
sb.heatmap(confusion_matrix(y_test, y_test_pred), 
           annot = True, fmt=".0f", annot_kws={"size": 18}, ax = axes[1])

Goodness of Fit of Model 	Train Dataset


NameError: name 'rforest' is not defined

## **Insights**

This model features much better performance than Linear Regression, with an R^2 value of 0.353, much higher than 0.18 for Model 1. Thus, for *Problem 1: Predicting Goals* we recommend using this random forest model in order to predict the number of goals socred in a match.

# Problem Formulation 2

Even if Random Forest is pretty okay at predicting goals from the other variables, the model may not be super useful to football players and football coaches - it may be used by Sports Data Analysts and Data Scientists, but not much beyond that. 

We want to find insights that are more useful to football teams to win football matches (that is why we first tried our feature importance approach).

Maybe there is some pattern in the data that we can't discern by eye?